# Estimation and Analysis of the ground-truth OD matrix

#### adding required packages

In [137]:
import pandas as pd
import numpy as np
import pickle
import time
import sys
# !conda install --yes --prefix {sys.prefix} pyproj
# !{sys.executable} -m pip install geopandas
from pyproj import Proj, transform
from shapely.geometry import Point
import geopandas as gpd
import random
from scipy.stats import gaussian_kde
import datetime as dt

#### specifying the saving location 

In [138]:
savingLoc = "Y:/ZahraEftekhar/phase4/"

#### functions for proximity analysis

Here, we create classes and functions to automate the proximity analysis. These analyses include changing the coordinate system, mapping the location coordinates to their associated TAZ (traffic analysis zone).

In [139]:
#this class change coordinate system and map location coordinates to zones:
class LongLat:
    def __init__(self, *args):
        self.TAZ = 0
    def set_location(self, x, y):
        from shapely.geometry import Point
        self.location = Point(x, y)

    def changeCoordSys(self, initial: str = 'epsg:23031', final: str = 'epsg:28992'):
        from pyproj import Proj, transform
        from shapely.geometry import Point
        self.location = Point(transform(Proj(init=initial), Proj(init=final), self.location.x, self.location.y))

    def zoneMapping(self, onepolygon, polygonName):
        if (onepolygon.contains(self.location)):
            self.TAZ = polygonName
# this class only reads SHP:
class TAZmap:
    def __init__(self): pass

    def set_map(self, value):
        import geopandas as gpd
        self.map = gpd.read_file(value).loc[:,['mzr_id','geometry']]

#### building the zero OD Matrix based on the number of zones in the SHP

In [140]:
# building the zero OD Matrix based on the number of zones in the SHP
map_mzr = TAZmap()
map_mzr.set_map("{a}amsterdamMezuroZones.shp".format(a=savingLoc))
inputs = map_mzr.map.geometry
amsterdamMezuroZones = pd.read_csv("{a}amsterdamMezuroZones.CSV".format(a=savingLoc), usecols=['mzr_id'])
tazNames = amsterdamMezuroZones['mzr_id']
zoneZero = pd.Series(0)
matrixRowColNames = tuple(zoneZero.append(tazNames))
odsize=len(matrixRowColNames)
del map_mzr
ODMatrix_df = pd.DataFrame(np.zeros((odsize, odsize), dtype=np.int32), columns=matrixRowColNames,
                           index=matrixRowColNames)  # creating empty OD matrix

## Specifying the OD matrix estimation interval
=========================================================

In [141]:
ODstart = "06:30:00"
ODend = "09:30:00"
startTime_OD = pd.to_timedelta(ODstart)
endTime_OD = pd.to_timedelta(ODend)

#### Reading the travel diaries to estimate the OD matrix

In [142]:
with open('{a}1.trueLocExperienced.pickle'.format(a=savingLoc), 'rb') as handle:
    itemlistExperienced = pickle.load(handle)

======================================================================================================

#### estimating the ground-truth OD matrix

In [ ]:
#we time the process
startTime = time.time()
for ID in itemlistExperienced.keys():#itemlistExperienced.keys() or ['100158'] or ['100048']
    activityListExperienced = itemlistExperienced[ID]
    activityListExperienced.loc[:,"end"].iloc[-1] = pd.to_timedelta(activityListExperienced.loc[:,"end"].iloc[-1]
                                                                  )+ pd.to_timedelta('24:00:00')
    j=1
    while j < len(np.arange(len(activityListExperienced))):
        if j==len(np.arange(len(activityListExperienced)))-1:
            start_time1 = pd.to_timedelta((activityListExperienced.loc[:,"start"]).iloc[j])
            end_time1 = pd.to_timedelta((activityListExperienced.loc[:,"end"]).iloc[j])
            start_time2 = pd.to_timedelta((activityListExperienced.loc[:,"start"]).iloc[0])+ pd.to_timedelta('24:00:00')
            endActivity = end_time1
            startNewActivity = start_time2
            if pd.to_timedelta('23:59:59')>=pd.to_timedelta(ODstart)>=pd.to_timedelta(start_time1):
                startTime_OD = pd.to_timedelta(ODstart)
            else:
                startTime_OD = pd.to_timedelta(ODstart) + pd.to_timedelta('24:00:00')
            if pd.to_timedelta('23:59:59')>= pd.to_timedelta(ODend) >=pd.to_timedelta(start_time1):
                endTime_OD =pd.to_timedelta(ODend)
            else:
                endTime_OD = pd.to_timedelta(ODend)+ pd.to_timedelta('24:00:00')
        else:
            start_time1 = pd.to_timedelta(activityListExperienced.loc[:,"start"].iloc[j])
            end_time1 = pd.to_timedelta(activityListExperienced.loc[:,"end"].iloc[j])
            start_time2 = pd.to_timedelta(activityListExperienced.loc[:,"start"].iloc[j+1])
            endActivity = end_time1 
            startNewActivity = start_time2 
        if pd.to_timedelta(start_time1) <= pd.to_timedelta(startTime_OD) < pd.to_timedelta(startNewActivity):
            if endTime_OD <= endActivity:
                break
            else:
                while pd.to_timedelta(endTime_OD) > pd.to_timedelta(endActivity):
                    point1 = LongLat()
                    point1.set_location(x=float(activityListExperienced.loc[:,"x"].iloc[j]),
                                        y=float(activityListExperienced.loc[:,"y"].iloc[j]))#*********ghablan:j-1
                    point1.changeCoordSys()
                    for k in range(len(tazNames)):
                        point1.zoneMapping(inputs[k], tazNames[k])
                    origin = point1.TAZ
                    point2 = LongLat()
                    if j == len(activityListExperienced) - 1:
                        point2.set_location(x=float(activityListExperienced.loc[:,"x"].iloc[0]),
                                            y=float(activityListExperienced.loc[:,"y"].iloc[0]))
                    else:
                        point2.set_location(x=float(activityListExperienced.loc[:,"x"].iloc[j+1]),
                                        y=float(activityListExperienced.loc[:,"y"].iloc[j+1])) #*****ghablan: j
                    point2.changeCoordSys()
                    for k in range(len(tazNames)):
                        point2.zoneMapping(inputs[k], tazNames[k])
                    destination = point2.TAZ
                    ODMatrix_df[origin][destination] = ODMatrix_df[origin][destination] + 1
                    j += 1
                    if j > len(np.arange(len(activityListExperienced))) - 1: break
                    if j== len(np.arange(len(activityListExperienced)))-1:
                        end_time1 = pd.to_timedelta((activityListExperienced.loc[:,"end"]).iloc[0])+ pd.to_timedelta('24:00:00')
                    else:
                        end_time1 = pd.to_timedelta((activityListExperienced.loc[:,"end"]).iloc[j])
                    endActivity = end_time1
                break
        j += 1
TXTFileName = "{a}OD({start1}-{start2}_{end1}-{end2}).pickle".format(a=savingLoc,start1 = ODstart[0:2],
                                                                                     start2 = ODstart[3:5],
                                                                     end1 = ODend[0:2], end2 = ODend[3:5])
with open(TXTFileName, 'wb') as handle:
    pickle.dump(ODMatrix_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
print((time.time() - startTime)//60,'minutes')

In [137]:
print(np.sum(np.sum(ODMatrix_df, axis=0)))

11948


================================================================================
### creating a `dict` file of all the trips that each user has in its travel diaries:

In [143]:
itemlistExperienced["1"]

,VEHICLE,activityType,x,y,start,end
0,1,work,629393.6676188618,5803949.115843206,06:52:04,16:49:20
0,1,home,632315.3322837545,5817000.086435355,17:10:47,06:29:59


In [66]:
#we time the process
startTime = time.time()

trips = {}
for ID in itemlistExperienced.keys():
    activities= itemlistExperienced[ID]
    activities.columns = ["VEHICLE","activityType","x","y","A_start","A_end"]
    activities["start"] = 0
    activities["duration"] = 0
    
    for j in np.arange(0,len(activities)):
            activities.loc[:,"start"].iloc[j] = pd.to_timedelta(activities.loc[:,"A_end"].iloc[j-1])
            activities.loc[:,"duration"].iloc[j]=pd.to_timedelta(activities.loc[:,"A_start"].iloc[j])-pd.to_timedelta(activities.loc[:,"A_end"].iloc[j-1])
    activities.drop(["activityType","x","y","A_start","A_end"],axis=1,inplace=True)
    trips[ID] = activities
TXTFileName = "{a}trips.pickle".format(a=savingLoc)
with open(TXTFileName, 'wb') as handle:
    pickle.dump(trips, handle, protocol=pickle.HIGHEST_PROTOCOL)

print((time.time() - startTime)//60,'minutes')

c:\programdata\anaconda3\envs\extractingodfromxml\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


1.0 minutes


In [70]:
trips["1"]

,VEHICLE,start,duration
0,1,0 days 06:29:59,0 days 00:22:05
0,1,0 days 16:49:20,0 days 00:21:27


### analysis of the ground truth location and activity type identification:
Here, we use Bayesian Theory to compute the probability of each even and activity, then infer the category depending on the temporal characteristics of each record (i.e starting time and duration). Remmember, we have to include Mezuro zone id instead of location coordinates to represent the spatial aggregation due to the fact that in empirical data the BTS location is reported, NOT the user's location. This only means that the diagonal of the OD matrix becomes zero.

In [10]:
with open('{a}1.trueLocExperienced.pickle'.format(a=savingLoc), 'rb') as handle:
    activities = pickle.load(handle)
for ID in activities.keys():
    activities[ID].loc[:,"end"].iloc[-1] = pd.to_timedelta(activities[ID].loc[:,"end"].iloc[-1]
                                                                  )+ pd.to_timedelta('24:00:00')
    activities[ID].loc[:,"duration"] =  pd.to_timedelta(activities[ID].loc[:,"end"])-pd.to_timedelta(activities[ID].loc[:,"start"])
#this file is the same as `1.trueLocExperienced.pickle` except that it also includes the duration of activities
TXTFileName = "{a}activities.pickle".format(a=savingLoc)
with open(TXTFileName, 'wb') as handle:
    pickle.dump(activities, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [144]:
with open("{a}trips.pickle".format(a=savingLoc),'rb') as handle:
    trips = pickle.load(handle)

In [12]:
#generating dataframe of trips and activities for the ease of operations and visualizability
activity_df = pd.DataFrame()
trip_df = pd.DataFrame()
for ID in activities.keys():
    activity_df = activity_df.append(activities[ID])
    trip_df = trip_df.append(trips[ID])
activity_df.reset_index(drop=True,inplace=True)
trip_df.reset_index(drop=True,inplace=True)
with open('{a}activity_df.pickle'.format(a=savingLoc), 'wb') as handle:
    pickle.dump(activity_df, handle, protocol=pickle.HIGHEST_PROTOCOL)    
with open('{a}trip_df.pickle'.format(a=savingLoc), 'wb') as handle:
    pickle.dump(trip_df, handle, protocol=pickle.HIGHEST_PROTOCOL)    

In [83]:
activity_df.head()

,VEHICLE,activityType,x,y,start,end,duration
0,1,work,629393.6676188618,5803949.115843206,06:52:04,16:49:20,09:57:16
1,1,home,632315.3322837545,5817000.086435355,17:10:47,1 days 06:29:59,13:19:12
2,10,work,627192.4415982522,5799465.4065392725,06:01:34,15:27:02,09:25:28
3,10,home,632315.3322837545,5817000.086435355,15:54:40,1 days 05:35:01,13:40:21
4,100007,work,634456.1049276257,5793373.482339734,06:32:44,15:59:53,09:27:09


Now, we generate __training sets__ from trips and activities. This includes the trips and activity of 1% of the users. each training record is the label, duration and starting time of that event/activity. Then we use the entire data as the __test set__ to identify the location/activity category. The location category is either `stay`(activity) or `pass-by`(trip). The activity categories are either `home`, `work` or `other`. In this step we compute the identification accuracy which is the number of correctly identified calss devided by the number of each class. For instance we compute `home` identification accuracy as follows:
$a = \frac{true-positive-`home`}{actual-number-`home`s } $

In [145]:
with open('{a}activity_df.pickle'.format(a=savingLoc), 'rb') as handle:
    activities = pickle.load(handle)
with open("{a}trip_df.pickle".format(a=savingLoc), 'rb') as handle:
    trips = pickle.load(handle)

In [146]:
#we time the process
startTime = time.time()

# _______________test seed based on user sampling _________________________________
seedSet = np.arange(101,151)
sensitivityTable = pd.DataFrame(index=seedSet)
sensitivityTable.index=seedSet
for s, seed in enumerate(seedSet):
    ids = pd.unique(activities.VEHICLE)
    random.seed(seed)
    indices = random.sample(range(len(ids)),round(.01*len(ids))) #1% sampling of users
    indices = pd.DataFrame(indices, columns=["VEHICLE"])
    trainIDs =pd.DataFrame(ids[indices])
    trainIDs.columns = ["VEHICLE"]
    trainingSet_trips = (trainIDs.merge(trips, on=["VEHICLE"], how="inner", sort=True,validate="1:m"))
    trainingSet_activities = (trainIDs.merge(activities, on=["VEHICLE"], how="inner", sort=True,validate="1:m"))
    with open('{a}/trainingActivity_seed{s}.pickle'.format(a=savingLoc,s=seed), 'wb') as handle:
        pickle.dump(trainingSet_activities, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{a}trainingTrip_seed{s}.pickle'.format(a=savingLoc,s=seed), 'wb') as handle:
        pickle.dump(trainingSet_trips, handle, protocol=pickle.HIGHEST_PROTOCOL)


    trainingSet_home = trainingSet_activities[trainingSet_activities.activityType == "home"]
    trainingSet_work = trainingSet_activities[trainingSet_activities.activityType == "work"]
    trainingSet_other = trainingSet_activities[(trainingSet_activities.activityType != "home") & (trainingSet_activities.activityType != "work")]
    trainingSet_home.to_csv("{a}trainingHome_seed{s}.CSV".format(a=savingLoc,s=seed),index=False,header=True)
    trainingSet_work.to_csv("{a}trainingWork_seed{s}.CSV".format(a=savingLoc,s=seed),index=False,header=True)
    trainingSet_other.to_csv("{a}trainingOther_seed{s}.CSV".format(a=savingLoc,s=seed),index=False,header=True)

    with open('{a}trainingHome_seed{s}.pickle'.format(a=savingLoc,s=seed), 'wb') as handle:
        pickle.dump(trainingSet_home, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{a}trainingWork_seed{s}.pickle'.format(a=savingLoc,s=seed), 'wb') as handle:
        pickle.dump(trainingSet_work, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{a}trainingOther_seed{s}.pickle'.format(a=savingLoc,s=seed), 'wb') as handle:
        pickle.dump(trainingSet_other, handle, protocol=pickle.HIGHEST_PROTOCOL)
    # ________________________________________________________________________________________
    # ****************************************************************************************
    home = activities[activities.activityType== "home"]
    work = activities[activities.activityType== "work"]
    other = activities[(activities.activityType != "home") & (activities.activityType != "work")]
    # ****************************************************************************************
    # ________________________ writing data: Trainset _________________________

    # ________________________________________________________________

    # ___________________________ Probability calculations_________________________________________
    #***************************************************************************
    prior_activity = len(activities)/(len(activities) + len(trips))
    prior_trip =  len(trips)/(len(activities) + len(trips))
    activities['activity?'] = np.log(prior_trip)+np.log(((gaussian_kde((
        pd.to_timedelta(trainingSet_trips.start)).dt.total_seconds()).pdf((
        pd.to_timedelta(activities.start)).dt.total_seconds()))))+np.log(((gaussian_kde(
        (pd.to_timedelta(trainingSet_trips.duration)).dt.total_seconds()).pdf((
        pd.to_timedelta(activities.duration)).dt.total_seconds()))))<=np.log(prior_activity)+np.log(((gaussian_kde((
        pd.to_timedelta(trainingSet_activities.start)).dt.total_seconds()).pdf((
        pd.to_timedelta(activities.start)).dt.total_seconds()))))+np.log(((gaussian_kde(
        (pd.to_timedelta(trainingSet_activities.duration)).dt.total_seconds()).pdf((
        pd.to_timedelta(activities.duration)).dt.total_seconds()))))
    sensitivityTable.loc[seed,"stay accuracy"] = sum(activities['activity?'])/len(activities)

    #***************************************************************************
    trips['trip?'] = np.log(prior_trip)+np.log(((gaussian_kde((
        pd.to_timedelta(trainingSet_trips.start)).dt.total_seconds()).pdf((
        pd.to_timedelta(trips.start)).dt.total_seconds()))))+np.log(((gaussian_kde(
        (pd.to_timedelta(trainingSet_trips.duration)).dt.total_seconds()).pdf((
        pd.to_timedelta(trips.duration)).dt.total_seconds()))))>=np.log(prior_activity)+np.log(((gaussian_kde((
        pd.to_timedelta(trainingSet_activities.start)).dt.total_seconds()).pdf((
        pd.to_timedelta(trips.start)).dt.total_seconds()))))+np.log(((gaussian_kde(
        (pd.to_timedelta(trainingSet_activities.duration)).dt.total_seconds()).pdf((
        pd.to_timedelta(trips.duration)).dt.total_seconds()))))
    sensitivityTable.loc[seed,"pass-by accuracy"] = sum(trips['trip?'])/len(trips)


    #***************************************************************************

    prior_home = len(trainingSet_home)/(len(trainingSet_home) + len(trainingSet_work) + len(trainingSet_other))
    prior_work =  len(trainingSet_work)/(len(trainingSet_home) + len(trainingSet_work) + len(trainingSet_other))
    prior_other = len(trainingSet_other)/(len(trainingSet_home) + len(trainingSet_work) + len(trainingSet_other))

    home['home'] = np.log(prior_home)+np.log(((gaussian_kde((
        pd.to_timedelta(trainingSet_home.start)).dt.total_seconds()).pdf((
        pd.to_timedelta(home.start)).dt.total_seconds()))))+np.log(((gaussian_kde(
        (pd.to_timedelta(trainingSet_home.duration)).dt.total_seconds()).pdf((
        pd.to_timedelta(home.duration)).dt.total_seconds()))))

    home['work'] = np.log(prior_work)+np.log(((gaussian_kde((
        pd.to_timedelta(trainingSet_work.start)).dt.total_seconds()).pdf((
        pd.to_timedelta(home.start)).dt.total_seconds()))))+np.log(((gaussian_kde(
        (pd.to_timedelta(trainingSet_work.duration)).dt.total_seconds()).pdf((
        pd.to_timedelta(home.duration)).dt.total_seconds()))))

    home['other'] = np.log(prior_other)+np.log(((gaussian_kde((
        pd.to_timedelta(trainingSet_other.start)).dt.total_seconds()).pdf((
        pd.to_timedelta(home.start)).dt.total_seconds()))))+np.log(((gaussian_kde(
        (pd.to_timedelta(trainingSet_other.duration)).dt.total_seconds()).pdf((
        pd.to_timedelta(home.duration)).dt.total_seconds()))))

    home['activity?'] = home[['home','work','other']].idxmax(axis=1)
#     (np.log(prior_home+((gaussian_kde((
#         pd.to_timedelta(trainingSet_home.start)).dt.total_seconds()).pdf((
#         pd.to_timedelta(home.start)).dt.total_seconds())))+((gaussian_kde(
#         (pd.to_timedelta(trainingSet_trips.duration)).dt.total_seconds()).pdf((
#         pd.to_timedelta(home.duration)).dt.total_seconds()))))>=np.log(prior_work+((gaussian_kde((
#         pd.to_timedelta(trainingSet_work.start)).dt.total_seconds()).pdf((
#         pd.to_timedelta(home.start)).dt.total_seconds())))+((gaussian_kde(
#         (pd.to_timedelta(trainingSet_work.duration)).dt.total_seconds()).pdf((
#         pd.to_timedelta(home.duration)).dt.total_seconds()))))) &(np.log(prior_home+((gaussian_kde((
#         pd.to_timedelta(trainingSet_home.start)).dt.total_seconds()).pdf((
#         pd.to_timedelta(home.start)).dt.total_seconds())))+((gaussian_kde(
#         (pd.to_timedelta(trainingSet_home.duration)).dt.total_seconds()).pdf((
#         pd.to_timedelta(home.duration)).dt.total_seconds()))))>=np.log(prior_other+((gaussian_kde((
#         pd.to_timedelta(trainingSet_other.start)).dt.total_seconds()).pdf((
#         pd.to_timedelta(home.start)).dt.total_seconds())))+((gaussian_kde(
#         (pd.to_timedelta(trainingSet_other.duration)).dt.total_seconds()).pdf((
#         pd.to_timedelta(home.duration)).dt.total_seconds())))))
    sensitivityTable.loc[seed,"home accuracy"] = sum(home['activity?']=="home")/len(home)


#***************************************************************************
    work['home'] = np.log(prior_home)+np.log(((gaussian_kde((
        pd.to_timedelta(trainingSet_home.start)).dt.total_seconds()).pdf((
        pd.to_timedelta(work.start)).dt.total_seconds()))))+np.log(((gaussian_kde(
        (pd.to_timedelta(trainingSet_home.duration)).dt.total_seconds()).pdf((
        pd.to_timedelta(work.duration)).dt.total_seconds()))))

    work['work'] = np.log(prior_work)+np.log(((gaussian_kde((
        pd.to_timedelta(trainingSet_work.start)).dt.total_seconds()).pdf((
        pd.to_timedelta(work.start)).dt.total_seconds()))))+np.log(((gaussian_kde(
        (pd.to_timedelta(trainingSet_work.duration)).dt.total_seconds()).pdf((
        pd.to_timedelta(work.duration)).dt.total_seconds()))))

    work['other'] = np.log(prior_other)+np.log(((gaussian_kde((
        pd.to_timedelta(trainingSet_other.start)).dt.total_seconds()).pdf((
        pd.to_timedelta(work.start)).dt.total_seconds()))))+np.log(((gaussian_kde(
        (pd.to_timedelta(trainingSet_other.duration)).dt.total_seconds()).pdf((
        pd.to_timedelta(work.duration)).dt.total_seconds()))))

    work['activity?'] = work[['home','work','other']].idxmax(axis=1)
    sensitivityTable.loc[seed,"work accuracy"] = sum(work['activity?']=="work")/len(work)

#***************************************************************************
    other['home'] = np.log(prior_home)+np.log(((gaussian_kde((
        pd.to_timedelta(trainingSet_home.start)).dt.total_seconds()).pdf((
        pd.to_timedelta(other.start)).dt.total_seconds()))))+np.log(((gaussian_kde(
        (pd.to_timedelta(trainingSet_home.duration)).dt.total_seconds()).pdf((
        pd.to_timedelta(other.duration)).dt.total_seconds()))))

    other['work'] = np.log(prior_work)+np.log(((gaussian_kde((
        pd.to_timedelta(trainingSet_work.start)).dt.total_seconds()).pdf((
        pd.to_timedelta(other.start)).dt.total_seconds()))))+np.log(((gaussian_kde(
        (pd.to_timedelta(trainingSet_work.duration)).dt.total_seconds()).pdf((
        pd.to_timedelta(other.duration)).dt.total_seconds()))))

    other['other'] = np.log(prior_other)+np.log(((gaussian_kde((
        pd.to_timedelta(trainingSet_other.start)).dt.total_seconds()).pdf((
        pd.to_timedelta(other.start)).dt.total_seconds()))))+np.log(((gaussian_kde(
        (pd.to_timedelta(trainingSet_other.duration)).dt.total_seconds()).pdf((
        pd.to_timedelta(other.duration)).dt.total_seconds()))))

    other['activity?'] = other[['home','work','other']].idxmax(axis=1)
    sensitivityTable.loc[seed,"other accuracy"] = sum(other['activity?']=="other")/len(other)

#     ***************************************************************************
    home.to_csv("{a}home_seed{s}.CSV".format(a=savingLoc,s=seed),index=False,header=True)
    work.to_csv("{a}work_seed{s}.CSV".format(a=savingLoc,s=seed),index=False,header=True)
    other.to_csv("{a}other_seed{s}.CSV".format(a=savingLoc,s=seed),index=False,header=True)

    with open('{a}home_seed{s}.pickle'.format(a=savingLoc,s=seed), 'wb') as handle:
        pickle.dump(home, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{a}work_seed{s}.pickle'.format(a=savingLoc,s=seed), 'wb') as handle:
        pickle.dump(work, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{a}other_seed{s}.pickle'.format(a=savingLoc,s=seed), 'wb') as handle:
        pickle.dump(other, handle, protocol=pickle.HIGHEST_PROTOCOL)
sensitivityTable.to_excel("{a}userSampling_onePercentLocationDetectionSensitivity.xlsx".format(a=savingLoc), header=True,
                          index=True)
print((time.time() - startTime)//60,'minutes')

C:\Users\ZAHRAE~1\AppData\Local\Temp/ipykernel_14136/3358508287.py:52: RuntimeWarning: divide by zero encountered in log
  pd.to_timedelta(activities.start)).dt.total_seconds()))))+np.log(((gaussian_kde(
C:\Users\ZAHRAE~1\AppData\Local\Temp/ipykernel_14136/3358508287.py:64: RuntimeWarning: divide by zero encountered in log
  pd.to_timedelta(trips.start)).dt.total_seconds()))))+np.log(((gaussian_kde(
C:\Users\ZAHRAE~1\AppData\Local\Temp/ipykernel_14136/3358508287.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home['home'] = np.log(prior_home)+np.log(((gaussian_kde((
C:\Users\ZAHRAE~1\AppData\Local\Temp/ipykernel_14136/3358508287.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

9.0 minutes


# Appendix

#### Generating spatial training sets with different proportionality to the entire data:

In [135]:
from pyproj import Proj, itransform

ids = pd.unique(activities.VEHICLE)
random.seed(101)
prop=0.10
indices = random.sample(range(len(ids)),round(prop*len(ids))) #1% sampling of users
indices = pd.DataFrame(indices, columns=["VEHICLE"])
trainIDs =pd.DataFrame(ids[indices])
trainIDs.columns = ["VEHICLE"]

# trainingSet_trips = (trainIDs.merge(trips, on=["VEHICLE"], how="inner", sort=True,validate="1:m"))
trainingSet_activities = (trainIDs.merge(activities, on=["VEHICLE"], how="inner", sort=True,validate="1:m"))

# building the zero OD Matrix based on the number of zones in the SHP
map_mzr = TAZmap()
map_mzr.set_map("{a}amsterdamMezuroZones.shp".format(a=savingLoc))
inputs = map_mzr.map.geometry
map_mzr.map
ids = pd.unique(activities.VEHICLE)
random.seed(101)
prop=0.10
indices = random.sample(range(len(ids)),round(prop*len(ids))) #1% sampling of users
indices = pd.DataFrame(indices, columns=["VEHICLE"])
trainIDs =pd.DataFrame(ids[indices])
trainIDs.columns = ["VEHICLE"]
trainingSet_trips = (trainIDs.merge(trips, on=["VEHICLE"], how="inner", sort=True,validate="1:m"))
trainingSet_activities = (trainIDs.merge(activities, on=["VEHICLE"], how="inner", sort=True,validate="1:m"))
points = list(zip([float(num) for num in (trainingSet_activities.x.values)],[float(num) for num in (trainingSet_activities.y.values)]))

# projection 1:
p1 = Proj(init='epsg:23031')
# projection 2: 
p2 = Proj(init='epsg:28992')
ptsX = []
ptsY = []
for pt in itransform(p1,p2,points, always_xy=True): 
    from shapely.geometry import Point
    ptsX = ptsX + [pt[0]]
    ptsY = ptsY + [pt[1]]

    
trainingSet_activities = gpd.GeoDataFrame(trainingSet_activities, geometry=gpd.points_from_xy(ptsX,ptsY))
trainingSet_activities = trainingSet_activities.set_crs(epsg=28992)
trainingSet_activities = gpd.sjoin(trainingSet_activities, map_mzr.map,how = "left")
trainingSet_activities.mzr_id = (np.nan_to_num((trainingSet_activities.mzr_id)))
trainingSet_activities.drop(['index_right'],axis=1,inplace=True)


trainingSet_home = trainingSet_activities[trainingSet_activities.activityType == "home"]
trainingSet_work = trainingSet_activities[trainingSet_activities.activityType == "work"]
trainingSet_other = trainingSet_activities[(trainingSet_activities.activityType != "home") & (trainingSet_activities.activityType != "work")]
trainingSet_home.to_csv("{a}trainingHome_prop{s}.CSV".format(a=savingLoc,s=prop),index=False,header=True)
trainingSet_work.to_csv("{a}trainingWork_prop{s}.CSV".format(a=savingLoc,s=prop),index=False,header=True)
trainingSet_other.to_csv("{a}trainingOther_prop{s}.CSV".format(a=savingLoc,s=prop),index=False,header=True)

C:\Users\zahraeftekhar\.conda\envs\ODmatrices\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\zahraeftekhar\.conda\envs\ODmatrices\lib\site-packages\pyproj\crs\crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
C:\Users\zahraeftekhar\.conda\envs\ODmatrices\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the

In [136]:
trainingSet_home

,VEHICLE,activityType,x,y,start,end,duration,geometry,mzr_id
1,10001,home,625308.5056096063,5792622.8888017945,17:00:03,1 days 06:57:56,0 days 13:57:53,POINT (117306.649 475676.791),7831.0
3,100089,home,630028.7167090132,5802955.596728954,22:27:16,1 days 20:20:51,0 days 21:53:35,POINT (122365.447 485849.465),7514.0
5,100294,home,630543.4389452781,5803946.491865685,21:53:42,1 days 17:23:16,0 days 19:29:34,POINT (122912.629 486822.948),7510.0
11,10052,home,625308.5056096063,5792622.8888017945,15:01:57,1 days 08:14:07,0 days 17:12:10,POINT (117306.649 475676.791),7831.0
13,100572,home,629997.9602208312,5803597.994578412,11:03:41,1 days 09:09:00,0 days 22:05:19,POINT (122355.893 486492.597),7510.0
...,...,...,...,...,...,...,...,...,...
5817,99721,home,628959.7643920127,5803554.357180867,16:11:41,1 days 06:29:18,0 days 14:17:37,POINT (121316.710 486483.224),7510.0
5819,9974,home,625308.5056096063,5792622.8888017945,17:10:09,1 days 07:11:11,0 days 14:01:02,POINT (117306.649 475676.791),7831.0
5822,99799,home,628774.5976465159,5802753.050154505,21:29:21,1 days 14:59:22,0 days 17:30:01,POINT (121105.193 485688.371),7513.0
5824,99876,home,628959.7643920127,5803554.357180867,17:46:42,19:34:52,0 days 01:48:10,POINT (121316.710 486483.224),7510.0
